In [1]:
%matplotlib inline

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets(
    'MNIST_data', one_hot=True, reshape=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
mnist.train.images.shape

(55000, 28, 28, 1)

In [4]:
with tf.Graph().as_default() as g:
    inputs = tf.placeholder(shape=[None, 28, 28, 1], dtype=tf.float32)
    labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)
    
    conv1_weight = tf.get_variable(
        'c1_w', [5, 5, 1, 8], dtype=tf.float32)  # h, w, s, c
    conv1_bias = tf.get_variable(
        'c1_b', [8], dtype=tf.float32)
    conv1_ = tf.nn.conv2d(
        inputs, conv1_weight, [1, 1, 1, 1], 'VALID')
    # shape = [None, 24, 24, 8]
    conv1 = tf.nn.relu(conv1_ + conv1_bias)
    
    # shape = [None, 12, 12, 8]
    pool1 = tf.nn.max_pool(
        conv1, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
    
    conv2_weight = tf.get_variable(
        'c2_w', shape=[5, 5, 8, 16], dtype=tf.float32)
    conv2_bias = tf.get_variable(
        'c2_b', shape=[16], dtype=tf.float32)
    conv2_ = tf.nn.conv2d(
        conv1_, conv2_weight, [1, 1, 1, 1], 'VALID')
    # shape = [None, 8, 8, 16]
    conv2 = tf.nn.relu(conv2_ + conv2_bias)
    
    # shape = [None, 4, 4, 16]
    pool2 = tf.nn.max_pool(
        conv2, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
    
    tmp = tf.reshape(pool2, [-1, 4 * 4 * 16])
    
#     pool2_shape= tf.shape(pool2)
#     tmp = tf.reshape(
#         pool2,
#         [-1, [pool2_shape[1] * pool2_shape[2]* pool2_shape[3]]])
    logits = tf.keras.layers.Dense(
        10, activation=None)(tmp)
    output = tf.nn.softmax(logits)
    
    saver = tf.train.Saver()
    
    # 代价函数
#     loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(label, logits)),
#         axis=1))
    loss = tf.reduce_mean(-1 * tf.reduce_sum(
        labels * tf.log(output + 1e-7),
        axis=1))
    
    
    # 正确率
    acc = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(labels, axis=1), tf.argmax(output, axis=1)),
                tf.float32))


with tf.Session(graph=g) as sess:
    # 定义梯度下降法优化器
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optim.minimize(loss)
    sess.run(tf.global_variables_initializer())
    
    # 训练模型
    for step in range(20000):
        batch_images, batch_labels = mnist.train.next_batch(32)
        res_loss, _ = sess.run([loss, train_op], feed_dict={
            inputs: batch_images,
            labels: batch_labels
        })
        
        # 输出代价并验证模型
        if step % 500 == 0:
            accs = []
            for test_step in range(10000 // 32):
                batch_images, batch_labels = mnist.test.next_batch(32)
                res_acc = sess.run(acc, feed_dict={
                    inputs: batch_images,
                    labels: batch_labels
                })
                accs.append(res_acc)
            accs = np.mean(accs)
            print('step %5d, loss %2.4f, acc %.4f' % (step, res_loss, accs))

step     0, loss 2.4042, acc 0.0907
step   500, loss 0.4672, acc 0.8620
step  1000, loss 0.3571, acc 0.9257
step  1500, loss 0.0399, acc 0.9441
step  2000, loss 0.3064, acc 0.9479
step  2500, loss 0.2009, acc 0.9531
step  3000, loss 0.1217, acc 0.9589
step  3500, loss 0.0769, acc 0.9639
step  4000, loss 0.0258, acc 0.9659
step  4500, loss 0.1755, acc 0.9691
step  5000, loss 0.1602, acc 0.9654
step  5500, loss 0.0535, acc 0.9680
step  6000, loss 0.0420, acc 0.9699
step  6500, loss 0.0886, acc 0.9729
step  7000, loss 0.0498, acc 0.9749
step  7500, loss 0.0179, acc 0.9742
step  8000, loss 0.2136, acc 0.9740
step  8500, loss 0.1258, acc 0.9760
step  9000, loss 0.0394, acc 0.9769
step  9500, loss 0.0945, acc 0.9768
step 10000, loss 0.0477, acc 0.9761
step 10500, loss 0.0293, acc 0.9771
step 11000, loss 0.0632, acc 0.9777
step 11500, loss 0.0846, acc 0.9795
step 12000, loss 0.0131, acc 0.9791


KeyboardInterrupt: 